In [81]:
#####ベイジアン変量効果二項回帰モデル#####
##みどり本10章:個体差と生存種子数
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy
from numpy.random import *
from scipy import optimize
from scipy.stats import norm

In [116]:
####データの発生####
#np.random.seed(8742)   #シードを設定

#データの設定
N = 1000   #観測個体数
n = poisson(15.0, N)   #個体ごとの種子数

In [117]:
##応答変数の生成
#パラメータの設定
tau = 0.4; taut = tau
alpha = alphat = normal(0, tau, N)   #変量効果
beta = betat = -0.5   #全体共通の切片

#ロジットモデルと確率を設定
logit = alpha + beta   #ロジット
Pr = np.exp(logit) / (1 + np.exp(logit))   #確率

#二項分布から応答変数を生成
y = binomial(n, Pr, N)
pd.DataFrame({'1. 種子数' : n, '2. 生存数' : y, '3. 観測確率' : y/n, '4. 真の確率' : Pr})   #データの確認

,1. 種子数,2. 生存数,3. 観測確率,4. 真の確率
0,525,277,0.527619,0.542702
1,506,254,0.501976,0.461910
2,490,158,0.322449,0.329487
3,506,253,0.500000,0.471898
4,510,125,0.245098,0.274834
5,510,257,0.503922,0.499500
6,531,183,0.344633,0.372150
7,508,234,0.460630,0.411454
8,503,153,0.304175,0.300486
9,492,127,0.258130,0.251829


In [118]:
####マルコフ連鎖モンテカルロ法で変量効果二項回帰モデルを推定####
##二項回帰モデルの対数尤度関数を設定
def define_likelihood(alpha, beta, n, y):
    #ロジットと確率の定義
    logit = alpha + beta
    Pr = np.exp(logit)/(1+np.exp(logit))
    
    #対数尤度の計算
    LLi = y * np.log(Pr) + (n-y) * np.log(1-Pr)
    LL = sum(LLi)   
    return LL, LLi

In [119]:
##アルゴリズムの設定
R = 10000
keep = 4  
iter = 0
burnin = int(2000/keep)
disp = 100

##事前分布の設定
tau1 = 100   #パラメータの事前分布
s0 = 0.01   #階層モデルの事前分布
v0 = 0.01
rw1 = 0.1   #ランダムウォーク幅
rw2 = 0.15

##初期値の設定
alpha = normal(0, 0.2, N)
beta = 0
tau2 = 0.5   #階層モデルの分散の初期値
inv_tau2 = 1/tau2

##パラメータの保存用配列
ALPHA = np.zeros((int(R/keep), N))
BETA = np.zeros((int(R/keep)))
TAU = np.zeros((int(R/keep)))
LL = np.zeros((int(R/keep)))

In [120]:
####メトロポリスヘイスティング法でパラメータをサンプリング####
for rp in range(R):
    ##全体共通のbetaをサンプリング
    #betaをサンプリング
    betad = beta
    betan = betad + normal(0, rw1)

    #対数尤度と対数事前分布の計算
    lognew1 = define_likelihood(alpha, betan, n, y)
    logold1 = define_likelihood(alpha, betad, n, y)
    logpnew1 = -0.5 * (np.power(betan, 2) / tau1)
    logpold1 = -0.5 * (np.power(betad, 2) / tau1)

    #MHサンプリングでbetaを採択するかどうかを決定
    gamma = min(1, np.exp(lognew1[0] + logpnew1 - logold1[0] - logpold1))
    u = uniform(0, 1, 1)

    #alpha > u なら新しいbetaを採択
    if gamma > u:
        beta = betan
        logl = lognew1
    else :
        beta = betad
        logl = logold1


    ##変量効果alphaをサンプリング
    #alphaをサンプリング
    alphad = alpha
    alphan = alphad + normal(0, rw2, N)

    #対数尤度と対数事前分布の計算
    lognew2 = define_likelihood(alphan, beta, n, y)
    logold2 = logl
    logpnew2 = -0.5 * (np.power(alphan, 2) / tau2)
    logpold2 = -0.5 * (np.power(alphad, 2) / tau2)

    #MHサンプリングでbetaを採択するかどうかを決定
    rand = uniform(0, 1, N)   #一様分布から乱数を発生
    LLind_diff = np.exp(lognew2[1] + logpnew2 - logold2[1] - logpold2)   #採択率
    gamma = (LLind_diff > 1)*1 + (LLind_diff <= 1)*LLind_diff

    #gammaの値に基づき新しいalphaを採択
    flag = (gamma >= rand)*1 + (gamma < rand)*0
    alpha = flag*alphan + (1-flag)*alphad


    ##逆ガンマ分布から階層モデルの分散をサンプリング
    s = s0 + sum(np.power(alpha - np.mean(alpha), 2))   
    v = v0 + N 
    tau2 = 1/np.random.gamma(v/2, 1/(s/2), 1)   #逆ガンマ分布からtauをサンプリング
    
    ##サンプリング結果の保存
    if rp%keep==0:
        mkeep = int(rp/keep)
        BETA[mkeep] = beta
        ALPHA[mkeep, :] = alpha
        TAU[mkeep] = tau2
        LL[mkeep] = logl[0]
    
    if rp%disp==0:
        print(rp)
        print(np.array([beta, tau2, ]))
        print(logl[0])

0
[-0.19734553  0.0374176 ]
-339562.1701006266
100
[-0.45418007  0.15926562]
-323254.7320893397
200
[-0.48127488  0.14800522]
-323243.1987720417
300
[-0.5012014   0.16204784]
-323222.3696428561
400
[-0.51570384  0.16088563]
-323282.68013917306
500
[-0.51600996  0.17501334]
-323279.3726656187
600
[-0.50124773  0.16177071]
-323282.88837673364
700
[-0.49756147  0.16740196]
-323254.77303578996
800
[-0.49477103  0.15562361]
-323246.3256002302
900
[-0.50326215  0.17474568]
-323248.4102415836
1000
[-0.49913051  0.17271803]
-323251.21512339497
1100
[-0.49810162  0.15814526]
-323248.7789775982
1200
[-0.49171257  0.16421008]
-323264.6464890557
1300
[-0.48913771  0.17597373]
-323243.49201909144
1400
[-0.49604136  0.16622094]
-323249.8922704449
1500
[-0.49870356  0.16097663]
-323212.82737828704
1600
[-0.49671475  0.16141197]
-323227.60223541886
1700
[-0.49424422  0.1611224 ]
-323215.3419868558
1800
[-0.48840222  0.15904455]
-323256.08292742504
1900
[-0.51220459  0.15821621]
-323245.39111392363
200

In [121]:
##サンプリング結果の要約
#サンプリング結果の事後平均
index = np.arange(burnin, int(R/keep))
beta = np.mean(BETA[index])
alpha = np.mean(ALPHA[index, ], axis=0)
tau = np.sqrt(np.mean(TAU[index]))
print([beta, tau])
print([betat, taut])

[-0.5076133046154299, 0.4064575118097931]
[-0.5, 0.4]


In [122]:
#推定された確率を計算
logit = alpha + beta   #ロジット
Prob = np.exp(logit) / (1 + np.exp(logit))

#データの確認
pd.DataFrame({'1. 種子数' : n, '2. 生存数' : y, '3. 変量効果' : alpha, '4. 真の変量効果' : alphat,  '5. 推定確率':Prob, 
              '6. 観測確率' : y/n, '7. 真の確率' : Pr})   

,1. 種子数,2. 生存数,3. 変量効果,4. 真の変量効果,5. 推定確率,6. 観測確率,7. 真の確率
0,525,277,0.590550,0.671227,0.520722,0.527619,0.542702
1,506,254,0.489983,0.347344,0.495593,0.501976,0.461910
2,490,158,-0.224400,-0.210506,0.324753,0.322449,0.329487
3,506,253,0.479369,0.387474,0.492939,0.500000,0.471898
4,510,125,-0.584800,-0.470232,0.251164,0.245098,0.274834
5,510,257,0.501760,0.498000,0.498537,0.503922,0.499500
6,531,183,-0.128955,-0.023004,0.346023,0.344633,0.372150
7,508,234,0.334664,0.142043,0.456870,0.460630,0.411454
8,503,153,-0.304361,-0.344982,0.307470,0.304175,0.300486
9,492,127,-0.516636,-0.588883,0.264201,0.258130,0.251829
